# Torch bridges examples

In [1]:
import copy

import numpy as np
from datasets import load_dataset
from plaid.bridges.huggingface_bridge import (
    huggingface_dataset_to_plaid,
    huggingface_description_to_problem_definition,
)
from plaid_ops.mesh.feature_engineering import update_dataset_with_sdf
from plaid_ops.mesh.transformations import (
    compute_bounding_box,
    project_on_regular_grid,
)
from torch.utils.data import DataLoader

# from plaid_bridges.torch.grid_fields_and_scalars import GridFieldsAndScalarsDataset

/home/fabien/miniconda3/envs/plaid-bridged/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Kokkos::OpenMP::initialize WARNING: OMP_PROC_BIND environment variable not set
  In general, for best performance with OpenMP 4.0 or better set OMP_PROC_BIND=spread and OMP_PLACES=threads
  For best performance with OpenMP 3.1 set OMP_PROC_BIND=true
  For unit testing set OMP_PROC_BIND=false



## Projection on constant rectilinear grid, with scalars as constant fields

In [2]:
hf_dataset = load_dataset(
    "PLAID-datasets/2D_Multiscale_Hyperelasticity", split="all_samples"
)
pb_def = huggingface_description_to_problem_definition(hf_dataset.info.description)
ids_train = pb_def.get_split("DOE_train")[:10]

dataset_train, _ = huggingface_dataset_to_plaid(
    hf_dataset, ids=ids_train, processes_number=5
)
n_train = len(dataset_train)
print(dataset_train)

Converting Hugging Face dataset to plaid dataset...


100%|██████████| 10/10 [00:00<00:00, 263.09it/s]

Dataset(10 samples, 4 scalars, 0 time_series, 7 fields)


In [3]:
dims = (101, 101)
dataset_train = update_dataset_with_sdf(dataset_train, verbose=True)

print(dataset_train.get_sample_ids())

bbox = compute_bounding_box(dataset_train)
projected_dataset_train = project_on_regular_grid(
    dataset_train, dimensions=dims, bbox=bbox, verbose=True
)

all_feat_ids = dataset_train[ids_train[0]].get_all_features_identifiers()
scalar_features = [f for f in all_feat_ids if "scalar" in f.values()]
field_features = [f for f in all_feat_ids if "field" in f.values()]

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 21.09it/s]


[376, 377, 378, 379, 380, 381, 382, 383, 384, 385]


100%|██████████| 10/10 [00:00<00:00, 32.65it/s]


In [4]:
# torch_dataset = GridFieldsAndScalarsDataset(
#     dataset=projected_dataset_train,
#     dimensions=dims,
#     in_features_identifiers=[field_features[0], scalar_features[0]],
#     out_features_identifiers=[scalar_features[1], field_features[1]],
# )

# loader = DataLoader(
#     torch_dataset,
#     batch_size=2,
#     shuffle=False,
# )

# out_feat_id = scalar_features[0]

# before = copy.deepcopy(
#     projected_dataset_train[ids_train[1]].get_feature_from_identifier(out_feat_id)
# )

# predictions = []
# for batch in loader:
#     for torch_sample in batch[1]:
#         predictions.append(torch_sample.detach().cpu().numpy())

# projected_dataset_train = torch_dataset.inverse_transform(predictions)

# after = copy.deepcopy(
#     projected_dataset_train[ids_train[1]].get_feature_from_identifier(out_feat_id)
# )

# print("Error after transform then inverse transform (2nd sample):")
# print(np.linalg.norm(after - before) / np.linalg.norm(before))

In [5]:
from plaid_bridges.common.base_regression_dataset import BaseRegressionDataset
from plaid_bridges.torch.grid_fields_and_scalars import (
    GridFieldsAndScalarsOfflineTransformer,
)

In [6]:
offline_transformer = GridFieldsAndScalarsOfflineTransformer(
    dimensions=dims,
    in_features_identifiers=[field_features[0], scalar_features[0]],
    out_features_identifiers=[scalar_features[1], field_features[1]],
)

In [7]:
torch_dataset = BaseRegressionDataset(
    dataset=projected_dataset_train,
    offline_transformer=offline_transformer,
)

In [8]:
loader = DataLoader(
    torch_dataset,
    batch_size=2,
    shuffle=False,
)

out_feat_id = scalar_features[0]

before = copy.deepcopy(
    projected_dataset_train[ids_train[1]].get_feature_from_identifier(out_feat_id)
)

predictions = []
for batch_x, batch_y in loader:
    for torch_sample in batch_y:
        predictions.append(torch_sample.detach().cpu())

pred_projected_dataset_train = offline_transformer.inverse_transform(
    projected_dataset_train, predictions
)

after = copy.deepcopy(
    pred_projected_dataset_train[ids_train[1]].get_feature_from_identifier(out_feat_id)
)

print("Error after transform then inverse transform (2nd sample):")
print(np.linalg.norm(after - before) / np.linalg.norm(before))

Error after transform then inverse transform (2nd sample):
0.0


In [9]:
print(torch_dataset)

RegressionDataset, initialized from plaid:Dataset(10 samples, 4 scalars, 0 time_series, 8 fields)


In [10]:
print(offline_transformer)
print(offline_transformer.show_details())

Transformer (2 input features, 2 output features)
Transformer: input features=['psi (field)', 'C11 (scalar)'], output features=['C12 (scalar)', 'P12 (field)']
